In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline

/home/guyue/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
COCO_YEAR = "2017"

# MS COCO Dataset
import coco
config = coco.CocoConfig()
config.display()

COCO_DIR =  os.path.join(ROOT_DIR, "dataset/coco")
model_dir = os.path.join(ROOT_DIR, "logs/")
model_path = os.path.join(ROOT_DIR, "model/mask_rcnn_coco.h5")


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  400
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [512 512   3]
KPS_ROI_POSITIVE_RATIO         0.33
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE          

In [4]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=model_dir)

In [5]:
# Load weights
print("Loading weights ", model_path)
model.load_weights(model_path, by_name=True)

Loading weights  /home/guyue/mywork/Mask_RCNN_keypoint/model/mask_rcnn_coco.h5


In [6]:
dataset_train = coco.CocoDataset()
dataset_train.load_coco(COCO_DIR, "train", year=COCO_YEAR)
dataset_train.prepare()


loading annotations into memory...
Done (t=11.49s)
creating index...
index created!
loading annotations into memory...
Done (t=5.58s)
creating index...
index created!


In [7]:
dataset_val = coco.CocoDataset()
val_type = "val" if COCO_YEAR in '2017' else "minival"
dataset_val.load_coco(COCO_DIR, val_type, year=COCO_YEAR)
dataset_val.prepare()

loading annotations into memory...
Done (t=0.34s)
creating index...
index created!
loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


In [ ]:
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=40,
            layers='heads')

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /home/guyue/mywork/Mask_RCNN_keypoint/logs/coco20181009T1607/mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4

/home/guyue/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/40
 349/1000 [=========>....................] - ETA: 2:20:01 - loss: 1.1659 - rpn_class_loss: 0.0324 - rpn_bbox_loss: 0.3749 - mrcnn_class_loss: 0.2554 - mrcnn_bbox_loss: 0.2190 - mrcnn_mask_loss: 0.2843